In [1]:
!git clone https://github.com/HoangHao1009/hminiGPT
%cd hminiGPT
!pip install -e .

Cloning into 'hminiGPT'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 121 (delta 28), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (121/121), 38.72 KiB | 2.42 MiB/s, done.
Resolving deltas: 100% (28/28), done.
/content/hminiGPT
Obtaining file:///content/hminiGPT
  Preparing metadata (setup.py) ... done
  Running setup.py develop for hhhGPT


In [2]:
pip install https://gitlab.com/trungtv/vi_spacy/-/raw/master/packages/vi_core_news_lg-3.6.0/dist/vi_core_news_lg-3.6.0.tar.gz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vi-core-news-lg: filename=vi_core_news_lg-3.6.0-py3-none-any.whl size=233275663 sha256=71606093a0ad152ee68205477cf8124fe37d427e093cf0588eec97e909bbe90c
  Stored in directory: /root/.cache/pip/wheels/bd/c2/22/8dfcbf9006c1be9c5f38dda2e8608eddb2f46c933f174c7581
Successfully built vi-core-news-lg


In [3]:
pip install pyvi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.4 MB/s eta 0:00:00


In [61]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import spacy
from hGPT.datacreator import DataCreator, Vocabulary, CustomDataset
from hGPT.model import MiniGPT, Trainer, WordVector
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
#model params
block_size = 128
batch_size = 32
n_emb = 512
num_head = 2
n_layer = 2
dropout = 0.3

cpu


In [7]:
checkpoint = torch.load('/content/drive/Othercomputers/My Laptop/Personal/hminiGPT/10000book/saveGPTmodel10000.pt', map_location = torch.device('cpu'))

In [10]:
checkpoint.keys()

dict_keys(['model', 'opt', 'voc'])

In [12]:
voc = Vocabulary()
voc.__dict__.update(checkpoint['voc'])

In [16]:
model = MiniGPT(voc.num_words, n_emb, block_size, num_head, n_layer, dropout, device)
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [17]:
wv = WordVector(voc, model, device)

Computing Word Vectos: 100%|██████████| 60644/60644 [00:02<00:00, 21859.47word/s]


In [32]:
wv.most_similarity('thông minh')

[('độ lượng', 0.2605756),
 ('lãng mạn', 0.25537658),
 ('chín chắn', 0.23102686),
 ('hồng hào', 0.23076029),
 ('đẹp trai', 0.23039979)]

In [35]:
wv.most_similarity('chó')

[('sói', 0.2891537),
 ('mèo', 0.2773193),
 ('thỏ', 0.2665468),
 ('voi', 0.2183971),
 ('dơi', 0.21670629)]

In [37]:
wv.similarity('mèo', 'chuột')

0.20588605

In [38]:
wv.similarity('điện thoại', 'tivi')

0.18364207

In [40]:
wv.similarity('lợi nhuận', 'doanh thu')

0.15295935

In [59]:
wv.similarity('vợ', 'chồng')

0.26509845

In [44]:
wv_loinhuan = wv.word_vector['lợi nhuận']
wv_doanhthu = wv.word_vector['doanh thu']
wv_chiphi = wv.word_vector['chi phí']

In [55]:
cosine_similarity([wv_loinhuan + wv_chiphi], [wv_doanhthu])[0][0]

0.17070982

In [56]:
wv_danong = wv.word_vector['đàn ông']
wv_danba = wv.word_vector['đàn bà']
wv_chong = wv.word_vector['chồng']
wv_vo = wv.word_vector['vợ']

In [60]:
cosine_similarity([wv_chong + wv_danong - wv_danba], [wv_vo])[0][0]

0.20854622

In [67]:
prompt = 'Ngày mai'
context = torch.tensor(voc.encode(prompt), dtype = torch.long, device = device)
generated_chars = voc.decode(model.generate(context.unsqueeze(0), max_new_tokens = 100)[0].tolist())
generated_chars = ' '.join([i for i in generated_chars.split() if i != 'UNK'])
print(generated_chars)

g à y m a i . l . bầu hànội . giây lát nghĩ rồi viết nghe nhiều tác giả nói : theo sở thích của de richelieu là dự đoán của , trở thành bạn của giới saint dear với draco . cho rồi mới phải trổ tài dan díu với kẻ giết những bạn bè . tất nhiên là con gái mình tranh luận , không phải lão ta chửi tai và cũng vẫn hoạt động luân hồi ) , đậu đen yêu quý rằng bác sĩ trùm một bên trong là cả ba matt chính sử , bị nguyễn công giáo lao theo kế hoạch cách mạng . là lực phát hiện . kế đó ! đại lượng của
